In [10]:
import pandas as pd
import tqdm
import os
import numpy as np
from gensim import models
a = [[[1,2], '1.2', '4.2'], [[1,2], '70', '0.03'], [[1,2], '5', '0']]
df = pd.DataFrame(a, columns=['one', 'two', 'three'])
df.head()

,one,two,three
0,"[1, 2]",1.2,4.2
1,"[1, 2]",70,0.03
2,"[1, 2]",5,0


In [12]:
import pandas as pd
import numpy as np
from gensim import models
from tqdm import tqdm
import os
def to_text_vector(words, model):
    # words = txt.split(',')
    array = np.asarray([model.wv[w] for w in words if w in words], dtype='float32')
    return array.mean(axis=0)


train_path = r"../data/train_fea/"
filenames = os.listdir(train_path)
filenames.sort(key=lambda x: int(x[6:8]))
train_data = []

'''
link id
link time
link ratio
link current status

这里link arrival status 没有被使用保存

cross id
cross time
'''
filenames = ["20200801_links.npz"]

100%|██████████| 1/1 [00:17<00:00, 17.72s/it]


links
20200801_links.npz


In [20]:
train_link_data = np.load(train_path + filenames[0])
train_rnn1 = train_link_data['data']
lkid_list1 = train_rnn1[:, :, 1].astype(int).tolist()  # his id

In [21]:
df = pd.DataFrame(lkid_list1)

In [22]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
0,0,0,0,0,0,0,0,0,0,0,...,3,2,4,11,16,2,1,0,1,9
1,0,0,0,0,0,0,0,0,0,0,...,10,31,4,8,7,10,5,5,1,19
2,0,0,0,0,0,0,0,0,0,0,...,8,4,40,6,9,1,7,27,10,2
3,0,0,0,0,0,0,0,0,0,0,...,2,3,12,1,5,3,1,2,7,0
4,0,0,0,0,0,0,0,0,0,0,...,5,14,3,8,6,8,5,4,1,7


In [ ]:
lkid_list1 = [str(list(filter(lambda x: x != 0, lkid))).strip('[').strip(']').replace(' ', '').split(',')
             for
             lkid in lkid_list1]

In [23]:

import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
import lightgbm as lgb
print(lgb.__version__)
import  time
import os
import tensorflow as tf
from tqdm import tqdm
train_path1 = r"../data/train_fea/20200801.csv"
train_path2 = r"../data/train_fea/20200802.csv"
test_path = r"../data/test_fea/20200901.csv"
submission_path = r"../sample_submission.csv"
train_path = r"../data/train_fea/"
filenames = os.listdir(train_path)
filenames.sort(key=lambda x: int(x[6:8]))
train_data = []
#filenames = ["20200801.csv","20200802.csv"]
for file in tqdm(filenames):
    try:
        train_data1 = pd.read_csv(train_path+file)
        train_data1['date'] = int(file[6:8])
        if len(train_data)==0:
            train_data = train_data1
        else:
            train_data = pd.concat([train_data, train_data1])
    except:
        pass

print(train_data.shape)
# submission = pd.read_csv(submission_path)
# train_data1 = pd.read_csv(train_path1)
# train_data1['date'] = 1
# train_data2 = pd.read_csv(train_path2)
# train_data2['date'] = 2

#train_data = pd.concat([train_data1,train_data2])
test_data =  pd.read_csv(test_path)
test_data['date'] = 32
data = pd.concat([train_data,test_data])
'''
所有特征
order_id,  订单id
ata,       实际行程时间（label）
distance,  订单距离
simple_eta,所有路段路口的平均时间和 （true_t+cross_t）
driver_id, 司机id
slice_id,  出发时刻
weather,   天气
hightemp,  最高气温
lowtemp,   最低气温
true_t,    所有link的平均时间和
wight_t,   计算路段通堵情况加权的link时间和
cross_t,   路口平均时间和
status_t,  状态和
link_num   link数量
cross_num  cross数量
'''
#print(train_data['ata'].describe())
# train_data = train_data[['order_id','ata','distance','simple_eta','driver_id','slice_id','weather','hightemp','lowtemp','true_t','wight_t','cross_t','status_t','link_num']]
# test_data = test_data[['order_id','ata','distance','simple_eta','driver_id','slice_id','weather','hightemp','lowtemp','true_t','wight_t','cross_t','status_t','link_num']]
#########################################单值扩展特征，包括均值和统计特征############################################
data['hour'] = data['slice_id'].apply(lambda x: x*5//60) #转化为小时
data['week_day'] = data['date'].apply(lambda x: x%7 + 1)
data['Temp_diff'] = data['hightemp']-data['lowtemp'] #温差
data['expect_velocity'] = data['distance']/data['simple_eta'] #预期速度
data['true_t_mean'] = data['true_t']/data['link_num'] # link的平均时间
data['wight_t_mean'] = data['wight_t']/data['link_num'] # 加权的link平均时间
data['status_t_mean'] = data['status_t']/data['link_num'] # 四种状态的平均状态
data['status_t_mean2'] = (data['state_num_1'] + 2*data['state_num_2'] + 3*data['state_num_3'])/3  #三种状态的平均状态(去除未知状态) (全为nan)
data['link_lens'] = data['distance']/data['link_num'] # link的平均长度
data['cross_t_mean'] = data['cross_t']/data['cross_num'] # 路口的平均通行时间
#比率特征
#state_ratio(全为nan)
# data['status_0_ratio'] = data['state_num_0']/(data['state_num_0'] + data['state_num_1'] + data['state_num_2'] + data['state_num_3'])
# data['status_1_ratio'] = data['state_num_1']/(data['state_num_0'] + data['state_num_1'] + data['state_num_2'] + data['state_num_3'])
# data['status_2_ratio'] = data['state_num_2']/(data['state_num_0'] + data['state_num_1'] + data['state_num_2'] + data['state_num_3'])
# data['status_3_ratio'] = data['state_num_3']/(data['state_num_0'] + data['state_num_1'] + data['state_num_2'] + data['state_num_3'])

# 不同路段每种状态下的平均通行时间
data['state_mean_time_0'] = data['state_sum_time_0']/data['state_num_0']
data['state_mean_time_1'] = data['state_sum_time_1']/data['state_num_1']
data['state_mean_time_2'] = data['state_sum_time_2']/data['state_num_2']
data['state_mean_time_3'] = data['state_sum_time_3']/data['state_num_3']
#time_ratio


#时序特征
# data["num_order_in_weather"] = data.groupby(["weather"])["order_id"].transform('count')
# data["weather_nums"] = data.groupby(["weather"])["date"].transform('nunique')
# data["order_nums_in_each_weather"] = data["num_order_in_weather"]/(data["weather_nums"])

#cross_link_ratio 全量直接提升5k?????
data['cross_t_ratio'] = data['cross_t']/data['simple_eta'] #cross时间占比
data['link_t_ratio'] = data['true_t']/data['simple_eta'] #link时间占比
# data['cross_link_ratio'] = data['true_t']/data['cross_t'] #cross/link时间占比
# data['cross_link_num_ratio'] = data['link_num']/data['cross_num'] #cross/link数量比
data['cross_num_ratio'] = data['cross_num']/(data['cross_num']+data['link_num']) #cross占比
data['link_num_ratio'] = data['link_num']/(data['cross_num']+data['link_num']) #link占比

3.2.1
(8651005, 24)


d:\softinstall\anaconda3\envs\tensorflow13\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\softinstall\anaconda3\envs\tensorflow13\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\softinstall\anaconda3\envs\tensorflow13\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\softinstall\anaconda3\envs\tensorflow13\lib\site-

In [24]:
data.head()

,order_id,ata,distance,simple_eta,driver_id,slice_id,weather,hightemp,lowtemp,true_t,...,link_lens,cross_t_mean,state_mean_time_0,state_mean_time_1,state_mean_time_2,state_mean_time_3,cross_t_ratio,link_t_ratio,cross_num_ratio,link_num_ratio
0,5966280,531,2923.0000,495,75457,153,5,28,26,368.0639,...,44.969231,25.40,1.000000,5.718750,NaN,NaN,0.256566,0.743563,0.071429,0.928571
1,6099942,590,3212.2780,559,19210,157,5,28,26,475.7070,...,50.988540,20.75,NaN,7.000000,13.6,7.0,0.148479,0.850996,0.059701,0.940299
2,2792816,559,7868.4474,626,38822,36,5,28,26,596.1600,...,126.910442,30.00,6.222222,10.169811,NaN,NaN,0.047923,0.952332,0.015873,0.984127
3,5780075,677,4976.3791,446,51627,76,5,28,26,414.2489,...,71.091130,32.00,NaN,5.914286,NaN,NaN,0.071749,0.928809,0.014085,0.985915
4,7808558,968,4985.9186,872,39965,52,5,28,26,754.9907,...,66.478915,23.40,14.000000,8.408163,8.5,NaN,0.134174,0.865815,0.062500,0.937500


In [54]:
test = data[(data['order_id']==7015369)]

test[['order_id','state_mean_time_1','cross_num','link_num']]

,order_id,state_mean_time_1,cross_num,link_num
332,7015369,9.133333,5,100


In [41]:
data['cross_num'].describe()


count    8.939081e+06
mean     6.199443e+00
std      4.140301e+00
min      1.000000e+00
25%      3.000000e+00
50%      5.000000e+00
75%      8.000000e+00
max      6.300000e+01
Name: cross_num, dtype: float64

In [42]:
data['simple_eta'].describe()

count    8.939081e+06
mean     7.482760e+02
std      4.656402e+02
min      2.000000e+00
25%      4.350000e+02
50%      6.360000e+02
75%      9.270000e+02
max      1.116100e+04
Name: simple_eta, dtype: float64